In [1]:
from lib.import_lang_data import *
from lib.lang_featurizers import *
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
# read data into lang_data, lang_results
languages = ['gcc', 'c', 'csharp', 'sbcl', 'clojure', 'ghc' 'java', 'javascript',
             'ocaml', 'perl', 'php', 'hack', 'py', 'python3', 'jruby', 'yarv', 'rb',
             'scala', 'racket']
lang_data, lang_results = read_polyglot(languages)
lang_info = pd.DataFrame(lang_results)
lang_info[0].value_counts()

gcc           58
scala         43
csharp        41
yarv          39
clojure       38
python3       36
ocaml         35
sbcl          34
perl          34
jruby         34
racket        29
php           29
hack          26
javascript    25
c              1
dtype: int64

In [3]:
lang_info = match_extensions(lang_info)
lang_results = list(lang_info[0])
lang_info[0].value_counts()

Ruby           73
C              59
PHP            55
Scala          43
C#             41
Clojure        38
Python         36
OCaml          35
Perl           34
Common Lisp    34
Scheme         29
JavaScript     25
dtype: int64

In [16]:
lang_featurizer = make_union(
    BagOfWordsFeaturizer(290),
    FunctionFeaturizer(presence_nil,
                       presence_nil_caps,
                       presence_null,
                       presence_none,
                       presence_start_double_semicolons,
                       presence_start_hashes,
                       presence_bar_hash,
                       percent_start_and_end_parenthesis,
                       presence_void,
                       presence_public,
                       presence_bool,
                       presence_module_line,
                       presence_extend_line,
                       presence_require_line,
                       presence_end,
                       presence_multiple_end,
                       presence_def_no_colon,
                       presence_at,
                       presence_double_at,
                       presence_defn,
                       percent_consecutive_closing_paren,
                       presence_from_import_line,
                       presence_import_line,
                       presence_print,
                       presence_dot_join,
                       presence_dot_format,
                       presence_dunder_name,
                       presence_dunder_init,
                       presence_def_colon,
                       
                       )
)

X_train, X_test, y_train, y_test = train_test_split(lang_data, lang_results)

pipe = make_pipeline(lang_featurizer, DecisionTreeClassifier())
# pipe = make_pipeline(lang_featurizer, MultinomialNB())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

1.0

In [17]:
print(classification_report(pipe.predict(X_test), y_test))

             precision    recall  f1-score   support

          C       1.00      1.00      1.00        13
         C#       1.00      1.00      1.00        10
    Clojure       1.00      1.00      1.00         6
Common Lisp       1.00      1.00      1.00         8
 JavaScript       1.00      1.00      1.00         4
      OCaml       1.00      1.00      1.00         8
        PHP       1.00      1.00      1.00        21
       Perl       1.00      1.00      1.00        11
     Python       1.00      1.00      1.00         5
       Ruby       1.00      1.00      1.00        22
      Scala       1.00      1.00      1.00        13
     Scheme       1.00      1.00      1.00         5

avg / total       1.00      1.00      1.00       126



## Now to test with the assignment tests

In [12]:
def read_tests():
    X = []
    y = None
    files = glob.glob('test/*')
    for file in files:
        with open(file) as f:
            X.append(f.read())

    with open('test.csv') as f:
        y = f.read()
    
    inter = y.split('\n')
    
    y_out = []
    for pair in inter:
        y_out.append(pair.split(','))
    
    y_out = list(pd.DataFrame(y_out).pop(1))[:-1]
    
    y_df = match_extensions(pd.DataFrame(y_out))
    y_out = list(y_df[0])
    
    return X, y_out

In [13]:
X_assignment_test, y_assignment_test = read_tests()

In [14]:
pipe.score(X_assignment_test, y_assignment_test)

0.0625

In [15]:
print(classification_report(pipe.predict(X_assignment_test), y_assignment_test))

             precision    recall  f1-score   support

          C       0.00      0.00      0.00         8
         C#       0.00      0.00      0.00         2
    Clojure       0.25      0.33      0.29         3
Common Lisp       0.00      0.00      0.00         1
    Haskell       0.00      0.00      0.00         0
       Java       0.00      0.00      0.00         0
 JavaScript       0.00      0.00      0.00         3
      OCaml       0.00      0.00      0.00         2
        PHP       0.33      0.14      0.20         7
     Python       0.00      0.00      0.00         0
       Ruby       0.00      0.00      0.00         3
      Scala       0.00      0.00      0.00         2
     Scheme       0.00      0.00      0.00         1
        TCL       0.00      0.00      0.00         0

avg / total       0.10      0.06      0.07        32



/Users/ahartz1/TIY/programming-language-classifier/.direnv/python-3.4.3/lib/python3.4/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/ahartz1/TIY/programming-language-classifier/.direnv/python-3.4.3/lib/python3.4/site-packages/sklearn/metrics/classification.py:960: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
